In [2]:
import scipy.io.idl as idl
import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [3]:
from scipy.spatial import ConvexHull, Delaunay
from scipy.interpolate import griddata

# Linearly interpolate $B_x$ at single timepoint in 3D

In [4]:
data_path = '../../comprehensive_3d_plot/output/2016-04-07/'
data_ending = '.sav'

bz_z249_file = data_path + 'bz_z249' + data_ending
bz_z302_file = data_path + 'bz_z302' + data_ending
bz_z357_file = data_path + 'bz_z357' + data_ending
bz_z416_file = data_path + 'bz_z416' + data_ending

bz_z249_measurements = idl.readsav(bz_z249_file)dd
bz_z302_measurements = idl.readsav(bz_z302_file)
bz_z357_measurements = idl.readsav(bz_z357_file)
bz_z416_measurements = idl.readsav(bz_z416_file)

measurements = {0.249: bz_z249_measurements, 
                0.302: bz_z302_measurements, 
                0.357: bz_z357_measurements, 
                0.416: bz_z416_measurements}

z_planes = [0.249, 0.302, 0.357, 0.416]

# First generate grid

In [5]:
time_point = 0
x_points = np.empty((0))
y_points = np.empty((0))
z_points = np.empty((0))
values = np.empty((0))
for z_plane in z_planes:
    plane_measurements = measurements[z_plane]
    x_points = np.append(x_points, plane_measurements['x_out'])
    y_points = np.append(y_points, plane_measurements['y_out'])
    z_points = np.append(z_points, np.ones(plane_measurements['x_out'].size)*z_plane)
    values = np.append(values, plane_measurements['a_out'][time_point])
    
    
points = [x_points, y_points, z_points]
points = np.asarray(points)

In [6]:
points = np.swapaxes(points, 0, 1)

In [7]:
points.shape

(4709, 3)

In [8]:
values.shape

(4709,)

In [11]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(points[:,0], points[:,1], points[:,2])
ax.set_ylabel('y')
ax.set_xlabel('x')
plt.show()

In [12]:
x_min, x_max = -0.027, 0.025
y_min, y_max = -0.021, 0.0295 
z_min, z_max = 0.249, 0.416
spatial_increment = 0.003

x_coord = np.linspace(x_min, x_max, np.ceil((x_max-x_min)/spatial_increment))
y_coord = np.linspace(y_min, y_max, np.ceil((y_max-y_min)/spatial_increment))
z_coord = np.linspace(z_min, z_max, np.ceil((z_max-z_min)/spatial_increment))

mesh = np.meshgrid(x_coord, y_coord, z_coord)

In [13]:
grid_points = np.dstack(map(np.ravel, mesh))[0]

# Now interpolate

In [14]:
interpolated_data = griddata(points, values, grid_points)

In [17]:
np.isnan(interpolated_data)

0

# Repeat interpolation for all three components of vector and with function calls

In [31]:
data_path = '../../comprehensive_3d_plot/output/2016-04-07/'
data_ending = '.sav'

bx_z249_file = data_path + 'bx_z249' + data_ending
bx_z302_file = data_path + 'bx_z302' + data_ending
bx_z357_file = data_path + 'bx_z357' + data_ending
bx_z416_file = data_path + 'bx_z416' + data_ending

bx_z249_measurements = idl.readsav(bx_z249_file)
bx_z302_measurements = idl.readsav(bx_z302_file)
bx_z357_measurements = idl.readsav(bx_z357_file)
bx_z416_measurements = idl.readsav(bx_z416_file)

measurements_x = {0.249: bx_z249_measurements, 
                  0.302: bx_z302_measurements, 
                  0.357: bx_z357_measurements, 
                  0.416: bx_z416_measurements}

z_planes = [0.249, 0.302, 0.357, 0.416]

In [32]:
data_path = '../../comprehensive_3d_plot/output/2016-04-07/'
data_ending = '.sav'

by_z249_file = data_path + 'by_z249' + data_ending
by_z302_file = data_path + 'by_z302' + data_ending
by_z357_file = data_path + 'by_z357' + data_ending
by_z416_file = data_path + 'by_z416' + data_ending

by_z249_measurements = idl.readsav(by_z249_file)
by_z302_measurements = idl.readsav(by_z302_file)
by_z357_measurements = idl.readsav(by_z357_file)
by_z416_measurements = idl.readsav(by_z416_file)

measurements_y = {0.249: by_z249_measurements, 
                  0.302: by_z302_measurements, 
                  0.357: by_z357_measurements, 
                  0.416: by_z416_measurements}

z_planes = [0.249, 0.302, 0.357, 0.416]

In [33]:
data_path = '../../comprehensive_3d_plot/output/2016-04-07/'
data_ending = '.sav'

bz_z249_file = data_path + 'by_z249' + data_ending
bz_z302_file = data_path + 'by_z302' + data_ending
bz_z357_file = data_path + 'by_z357' + data_ending
bz_z416_file = data_path + 'by_z416' + data_ending

bz_z249_measurements = idl.readsav(bz_z249_file)
bz_z302_measurements = idl.readsav(bz_z302_file)
bz_z357_measurements = idl.readsav(bz_z357_file)
bz_z416_measurements = idl.readsav(bz_z416_file)

measurements_z = {0.249: bz_z249_measurements, 
                  0.302: bz_z302_measurements, 
                  0.357: bz_z357_measurements, 
                  0.416: bz_z416_measurements}

z_planes = [0.249, 0.302, 0.357, 0.416]

In [34]:
direction_measurements = [measurements_z, measurements_y, measurements_z] 

In [35]:
def read_points_from_measurement_dict(measurement_dict, time_point, z_planes):
    x_points = np.empty((0))
    y_points = np.empty((0))
    z_points = np.empty((0))
    values = np.empty((0))
    for z_plane in z_planes:
        plane_measurements = measurements[z_plane]
        x_points = np.append(x_points, plane_measurements['x_out'])
        y_points = np.append(y_points, plane_measurements['y_out'])
        z_points = np.append(z_points, np.ones(plane_measurements['x_out'].size)*z_plane)
        values = np.append(values, plane_measurements['a_out'][time_point])
    
    
    points = [x_points, y_points, z_points]
    points = np.asarray(points)
    points = np.swapaxes(points, 0, 1)
    return points, values

In [38]:
points = []
values = []
for measurements in direction_measurements:
    points_direction, values_direction = read_points_from_measurement_dict(measurements, 0, z_planes)
    points.append(points_direction)
    values.append(values_direction)

In [73]:
def mesh(bounds, vector_points, spatial_increment):
    
    (x_min, x_max), (y_min, y_max), (z_min, z_max) = bounds
    
    x_coord = np.linspace(x_min, x_max, np.ceil((x_max-x_min)/spatial_increment))
    y_coord = np.linspace(y_min, y_max, np.ceil((y_max-y_min)/spatial_increment))
    z_coord = np.linspace(z_min, z_max, np.ceil((z_max-z_min)/spatial_increment))
    
    sizes = map(np.size, [x_coord, y_coord, z_coord])

    mesh = np.meshgrid(x_coord, y_coord, z_coord, indexing='ij')
    
    grid_points = np.dstack(map(np.ravel, mesh))[0]
    
    print np.allclose(grid_points[:,0].reshape(sizes), mesh[0])
    print np.allclose(grid_points[:,1].reshape(sizes), mesh[1])
    print np.allclose(grid_points[:,2].reshape(sizes), mesh[2])
    
    return grid_points, sizes

In [74]:
x_min, x_max = -0.027, 0.025
y_min, y_max = -0.021, 0.0295 
z_min, z_max = 0.249, 0.416
bounds = ((x_min, x_max), (y_min, y_max), (z_min, z_max)) 

grid_points, sizes = mesh(bounds, points, 0.003)

True
True
True


In [204]:
def interpolate(grid_points, vector_points, vector_values):
    interpolated_data_x = griddata(vector_points[0], vector_values[0], grid_points)
    interpolated_data_y = griddata(vector_points[1], vector_values[1], grid_points)
    interpolated_data_z = griddata(vector_points[2], vector_values[2], grid_points)
    return [interpolated_data_x, interpolated_data_y, interpolated_data_z]

In [238]:
interpolated_vector = interpolate(grid_points, points, values)

In [194]:
from pyvisfile.vtk import write_structured_grid
from pytools.obj_array import make_obj_array

In [196]:
test0 = grid_points[:,0].reshape(sizes)
test1 = grid_points[:,1].reshape(sizes)
test2 = grid_points[:,2].reshape(sizes)
test_grid = np.asarray([test0, test1, test2])

In [197]:
np.allclose(grid_points.swapaxes(0,1).reshape((3, sizes[0], sizes[1], sizes[2])), test_grid)

True

In [198]:
vtk_grid = grid_points.swapaxes(0,1).reshape((3, sizes[0], sizes[1], sizes[2]))

In [247]:

vtk_vector_x = np.resize(interpolated_vector[0], sizes)
vtk_vector_y = np.resize(interpolated_vector[1], sizes)
vtk_vector_z = np.resize(interpolated_vector[2], sizes)

vtk_vector_x = np.expand_dims(vtk_vector_x, 0)
vtk_vector_y = np.expand_dims(vtk_vector_y, 0)
vtk_vector_z = np.expand_dims(vtk_vector_z, 0)

vtk_vector = make_obj_array([vtk_vector_x, vtk_vector_y, vtk_vector_z])

In [248]:
vtk_grid.shape

(3, 18, 17, 56)

In [249]:
vtk_vector[0].shape

(1, 18, 17, 56)

In [251]:
write_structured_grid('test.vts', vtk_grid, point_data=[('B', vtk_vector)])

RuntimeError: output file 'test.vts' already exists